In [ ]:
import os
import re
import pandas as pd
from openai import OpenAI
from canvasapi import Canvas
from canvasapi.course import Course
from canvasapi.assignment import Assignment
from dotenv import load_dotenv

load_dotenv()

# Initialize a new Canvas object
canvas = Canvas(os.environ.get("CANVAS_API_URL"), os.environ.get("CANVAS_API_KEY_0"))
# Initialize a new OpenAI object
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
n_users = 1

In [ ]:
user = canvas.get_user(os.environ.get("CANVAS_USER_ID_0"))
df = pd.DataFrame(columns=["course", "assignmentid", "assignmenttype", "completion"])

In [ ]:
user.id

In [ ]:
canvas.get_courses()[0].id

In [ ]:
for course in canvas.get_courses():
    for assignment in course.get_assignments():
        if hasattr(assignment, "discussion_topic"):
            assignment_type="discussion"
        elif hasattr(assignment, "quiz_id"):
            assignment_type="quiz"
        else:
            assignment_type=None
        
        record = pd.DataFrame(
            {
                'course': [course.id],
                'id': [assignment.id],
                'assignmenttype': [assignment_type],
                'completion': [0]
            }
        )
        df = pd.concat([df, record], ignore_index=True)

In [ ]:
df

In [ ]:
for i in range(0, df.shape[0]):
    if df['completion'][i] == 0:
        course = canvas.get_course(df['course'][i])
        assignment = course.get_assignment(df['assignmentid'][i])
        assignment_type = df['assignmenttype'][i]

        if assignment_type == 'discussion':
            discussion_topic = course.get_discussion_topic(assignment.discussion_topic["id"])
            topic = discussion_topic.message
            images = re.findall(r"(?<=img src=\").*?(?=\")", topic)

            content = [
                {
                    "type": "text", 
                    "text": "Respond with the context that you are a community college student.\n\n" +
                            f"Additional information: my name is {user.name}\n\n" +
                            f"Question data: {topic}\n\n" + 
                            "Interpret the image(s) that came with the post as well."
                },
            ]
            for image in images:
                content.append(
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image,
                        },
                    },
                )

            response = openai_client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {
                    "role": "user",
                    "content": content
                    }
                ]
            )

            print(response.choices[0])
            # discussion_topic.post_entry(message=chat_completion.choices[0])



In [ ]:
import os
import re
import datetime
import pytz

import click
from dotenv import load_dotenv
from canvasapi import Canvas
from canvasapi.module import Module, ModuleItem
from canvasapi.discussion_topic import DiscussionTopic

load_dotenv()

canvas = Canvas(os.environ["API_URL"], os.environ["API_KEY"])

# CIS_10 = [43977, 43876]
# CIS_80 = [44572, 43032]
@click.command()
@click.option("--course_id", required=True, type=int)
@click.option("-y", is_flag=True)
def main(course_id, y):
    course = canvas.get_course(course_id)
    print(course)
    
    if not y:
        print("You MUST have a due date (mm/dd) in the modules' title that you want to modify with this script")
        confirmation = input("Module titles have correct due dates (Y/n): ")
        if confirmation.upper() != "Y" and confirmation.upper() != "":
            exit()

    for module in course.get_modules():
        module: Module
        due_date = re.search("(\d+)\\/(\d+)", module.name)
        # handle modules with listed due dates
        if due_date and len(due_date.regs) == 3:
            # convert due date to datetime in UTC
            now = datetime.datetime.now()
            due_datetime = datetime.datetime(now.year, int(due_date.group(1)), int(due_date.group(2)), 23, 59, 59, 0)
            due_datetime = pytz.timezone("US/Pacific").localize(due_datetime)
            unlock_datetime = due_datetime - datetime.timedelta(days=10)
            # need to replace hour, minute and second because DST makes the unlock date to drift
            unlock_datetime = unlock_datetime.replace(hour=23, minute=59, second=59)
            # need to replace hour, minute and second because DST makes the due date to drift
            due_datetime = due_datetime.astimezone(pytz.timezone("UTC"))
            
            print(f"{module} due date: {due_datetime} UTC")

            # set module unlock 10 days before due
            print(f"module unlock date: {unlock_datetime} Pacific")

            unlock_at_canvas_formatted_iso_string = unlock_datetime.isoformat().split("+")[0] + "Z"
            if unlock_at_canvas_formatted_iso_string != module.unlock_at:
                updated_module = module.edit(module={
                    "unlock_at": unlock_datetime
                })
                print(f"updated {updated_module.name} with unlock date {updated_module.unlock_at}")
            
            # loop over module items, and publish
            for current_module_item in module.get_module_items():
                current_module_item: ModuleItem
                print("\t" + str(current_module_item))

                if not current_module_item.published:
                    if current_module_item.type == "Page" and "DO NOT PUBLISH" not in current_module_item.title:
                        current_module_item.edit(module_item={
                            "published": "true"
                        })
                    elif current_module_item.type == "ExternalTool":
                        current_module_item.edit(module_item={
                            "published": "true"
                        })
                    elif current_module_item.type == "SubHeader":
                        current_module_item.edit(module_item={
                            "published": "true"
                        })

                if current_module_item.type == "Assignment": # for each assignment, set due as indicated in module title
                    assignment = course.get_assignment(current_module_item.content_id)
                    print(f"\tdue date {assignment.due_at} and lock date {assignment.due_at}")

                    due_at_canvas_formatted_iso_string = due_datetime.isoformat().split("+")[0] + "Z"
                    if assignment.due_at != due_at_canvas_formatted_iso_string:
                        updated_assignment = assignment.edit(assignment={
                            "due_at": due_datetime,
                            "lock_at": due_datetime,
                            "published": "true"
                        })
                        if updated_assignment.due_at not in updated_assignment.lock_at:
                            print("PROBLEM")
                        print(f"updated {updated_assignment.name} with due date {updated_assignment.due_at} and lock date {updated_assignment.due_at}")
                elif current_module_item.type == "Quiz": # for each quiz, set due as indicated in module title
                    quiz = course.get_quiz(current_module_item.content_id)
                    print(f"\tdue date {quiz.due_at} and lock date {quiz.due_at}")

                    due_at_canvas_formatted_iso_string = due_datetime.isoformat().split("+")[0] + "Z"
                    if quiz.due_at != due_at_canvas_formatted_iso_string:
                        updated_quiz = quiz.edit(quiz={
                            "due_at": due_datetime,
                            "lock_at": due_datetime,
                            "published": "true"
                        })
                        print(f"updated {updated_quiz.title} with due date {updated_quiz.due_at} and lock date {updated_quiz.due_at}")


if __name__ == "__main__":
    main()

In [ ]:
df

In [ ]:
for item in course.get_discussion_topics():
    print(item)

course.get_discussion_topics

In [ ]:
course = canvas.get_course(df['course'][0])
assignment = course.get_assignment(df['assignmentid'][0])
# assignment_type = df['assignmenttype'][0]
assignment.quiz_id
# assignment.discussion_topic['id']
quiz = course.get_quiz(assignment.quiz_id)

for q in quiz.get_questions():
    print(dir(q))
    print(q.question_text)
    print(q.question_type)
    print(q.set_attributes)
    print(q.variables)
    print(q.answers)


In [ ]:
response.choices[0].message.content

In [ ]:
for i in range(0, 3):
    {
        "test": i
    },

In [ ]:
discussion_topic.post_entry(message=chat_completion.choices[0].message.content)

In [ ]:
discussion_topic.message, chat_completion.choices[0].message.content